# how to get related words by word2vec?

## Search Tree -> Similar Words

In [1]:
csv_path = '/home/zhaodao/桌面/github/NLP_study/第二次内容/datasource/export_sql_1558435.zip'

In [2]:
import pandas as pd


In [3]:
content = pd.read_csv(csv_path, encoding='gb18030')

In [4]:
content = content.fillna('')

In [5]:
news_content = content['content'].tolist()

In [6]:
import jieba

In [7]:
def cut(string):
    return ' '.join(jieba.cut(string))

In [8]:
cut('这是一个测试')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.409 seconds.
Prefix dict has been built succesfully.


'这是 一个 测试'

In [9]:
import re

In [10]:
def token(string):
    return re.findall(r'[\d|\w]+', string)

In [11]:
token('这是一个测试\n\n\n')

['这是一个测试']

In [12]:
news_content = [token(n) for n in news_content]

In [13]:
news_content = [' '.join(n) for n in news_content]

In [14]:
news_content = [cut(n) for n in news_content]

In [15]:
len(news_content)

89611

In [16]:
news_content[1]

'骁龙 835 作为 唯一 通过 Windows   10 桌面 平台 认证 的 ARM 处理器   高通 强调   不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心   相反   他们 正 联手 微软   找到 一种 适合 桌面 平台 的   兼顾 性能 和 功耗 的 完美 方案   报道 称   微软 已经 拿到 了 一些 新 的 源码   以便 Windows   10 更好 地 理解 big   little 架构   资料 显示   骁龙 835 作为 一款 集成 了 CPU   GPU   基带   蓝牙   Wi   Fi 的 SoC   比 传统 的 Wintel 方案 可以 节省 至少 30   的 PCB 空间   按计划   今年 Q4   华硕   惠普   联想 将 首发 骁龙 835   Win10 电脑   预计 均 是 二合一 形态 的 产品   当然   高通 骁龙 只是 个 开始   未来 也许 还 能 见到 三星 Exynos   联发科   华为 麒麟   小米 澎湃 等 进入 Windows   10 桌面 平台'

In [17]:
with open('news-sentences-cut.txt', 'w') as f:
    for n in news_content:
        f.write(n + '\n')

In [19]:
from gensim.models import Word2Vec

In [20]:
from gensim.models.word2vec import LineSentence

In [21]:
news_word2ve = Word2Vec(LineSentence('news-sentences-cut.txt'), size=35, workers=8)

In [22]:
news_word2ve.most_similar('葡萄牙')

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('意大利', 0.8953498601913452),
 ('捷克', 0.8746763467788696),
 ('克罗地亚', 0.8579184412956238),
 ('摩洛哥', 0.8542280197143555),
 ('瑞士', 0.8326007723808289),
 ('巴塞罗那', 0.8307761549949646),
 ('多哈', 0.8224089741706848),
 ('比利时', 0.8209236264228821),
 ('丹麦', 0.8152623772621155),
 ('西班牙', 0.8117986917495728)]

## More Date, Better Results

1. 分词的问题
2. **数据量** 数据越多，效果越好

In [23]:
news_word2ve.most_similar('说')

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('表示', 0.8999356031417847),
 ('指出', 0.8666229248046875),
 ('认为', 0.854762077331543),
 ('坦言', 0.8156627416610718),
 ('看来', 0.7983412146568298),
 ('介绍', 0.7638407945632935),
 ('明说', 0.7625210285186768),
 ('文说', 0.7567147016525269),
 ('告诉', 0.7466958165168762),
 ('透露', 0.7245856523513794)]

In [24]:
news_word2ve.most_similar('表示')

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('指出', 0.9132236838340759),
 ('说', 0.8999356031417847),
 ('认为', 0.8779436349868774),
 ('透露', 0.8141977190971375),
 ('坦言', 0.802665650844574),
 ('强调', 0.7953732013702393),
 ('介绍', 0.7485463619232178),
 ('看来', 0.7436285018920898),
 ('称', 0.7149076461791992),
 ('建议', 0.6876084804534912)]

In [25]:
news_word2ve.most_similar('建议')

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('认为', 0.7457494735717773),
 ('指出', 0.7184297442436218),
 ('要求', 0.7008938789367676),
 ('表示', 0.6876084804534912),
 ('强调', 0.6756607890129089),
 ('呼吁', 0.6513333916664124),
 ('解释', 0.6433683633804321),
 ('意见建议', 0.6357353329658508),
 ('评价', 0.632862389087677),
 ('鼓励', 0.6282050013542175)]

# 根据已有的词汇，得到更多的词汇

In [26]:
from collections import defaultdict

In [37]:
def get_related_words(initial_words, model):
    """
    @initial_words are initial words we already know
    @model is the word2vec model
    """

    unseen = initial_words
    
    seen = defaultdict(int)
    
    max_size = 500
    
    while unseen and len(seen) < max_size:
        if len(seen) % 50 == 0:
            print('seen length : {}'.format(len(seen)))
        node = unseen.pop(0)
        
        new_expanding = [w for w, s in model.most_similar(node, topn=20)]
        
        unseen += new_expanding
        
        seen[node] += 1 # 打分机制，可以优化
        
        ## 优化点：
        #优化点1. score function could be revised
        #优化点2. using dymanic programming to reduce computing time
    return seen

In [39]:
related_words = get_related_words(['说', '表示'], news_word2ve)

seen length : 0
seen length : 50


/usr/lib/python3/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


seen length : 100
seen length : 100
seen length : 100
seen length : 150
seen length : 150
seen length : 200
seen length : 200
seen length : 250
seen length : 300
seen length : 350
seen length : 350
seen length : 400
seen length : 400
seen length : 450


In [43]:
sorted(related_words.items(), key=lambda x:x[1], reverse=True)

[('表示', 89),
 ('指出', 88),
 ('说', 87),
 ('认为', 79),
 ('坦言', 69),
 ('透露', 64),
 ('所说', 52),
 ('看来', 50),
 ('明说', 50),
 ('告诉', 50),
 ('介绍', 47),
 ('文说', 43),
 ('强调', 41),
 ('提到', 41),
 ('称', 40),
 ('中说', 34),
 ('建议', 34),
 ('呼吁', 28),
 ('普遍认为', 28),
 ('写道', 26),
 ('提及', 25),
 ('申明', 23),
 ('特别强调', 23),
 ('中称', 21),
 ('觉得', 19),
 ('时说', 17),
 ('说道', 17),
 ('声称', 17),
 ('而言', 16),
 ('地说', 16),
 ('问', 16),
 ('深有体会', 16),
 ('来说', 15),
 ('相信', 13),
 ('事实上', 13),
 ('直言', 13),
 ('如是说', 12),
 ('看法', 11),
 ('表明', 11),
 ('但是', 11),
 ('重申', 11),
 ('敦促', 11),
 ('承认', 11),
 ('希望', 11),
 ('还是', 10),
 ('还称', 10),
 ('说法', 10),
 ('引用', 10),
 ('知道', 10),
 ('回答', 10),
 ('原话', 10),
 ('描述', 10),
 ('解释', 10),
 ('坦承', 9),
 ('评述', 9),
 ('对此', 9),
 ('批评', 9),
 ('谈到', 8),
 ('当然', 8),
 ('道', 7),
 ('管理司', 7),
 ('知情', 7),
 ('推测', 7),
 ('反复强调', 7),
 ('要说', 7),
 ('上称', 7),
 ('时称', 7),
 ('其实', 7),
 ('叙林', 6),
 ('条法司', 6),
 ('了然于心', 6),
 ('眼中', 6),
 ('资深', 6),
 ('了解', 6),
 ('获悉', 6),
 ('中山大学', 6),
 ('旅游局', 6),
 ('供图', 6)

# 弱监督　＋启发式搜索　可以解决很多问题

向量化效果是关键。

向量化　就是　表征（reprezation）

如何将词向量化，如何将人向量化（社交媒体中）

# keywords 关键字
## which words are important?
1. 文本中出现的频率高
2. 在本文中出现的频率高，但是在其他的场合出现的频率就很少

## TF-IDF
TF: Term Frequency , 即在文章中出现的频率

DF: 包含单词T的文章的个数。　值越大，表明Ｔ的重要性越低

idf: idf(t) = log(N/(df(t)))

tf-idf = tf * idf


## TF-IDF Key words

In [46]:
news_content[0]

'此外   自 本周   6 月 12 日   起   除 小米 手机 6 等 15 款 机型 外   其余 机型 已 暂停 更新 发布   含 开发 版   体验版 内测   稳定版 暂不受 影响   以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作   有人 猜测 这 也 是 将 精力 主要 用到 MIUI   9 的 研发 之中   MIUI   8 去年 5 月 发布   距今已有 一年 有余   也 是 时候 更新换代 了   当然   关于 MIUI   9 的 确切 信息   我们 还是 等待 官方消息'

In [47]:
def document_frequency(word):
    return sum(1 for n in news_content if word in n)

In [48]:
document_frequency('关于')

3376

In [49]:
import math

In [50]:
def idf(word):
    """We get the inversed document frequency"""
    return math.log10(len(news_content) / document_frequency(word))

In [53]:
idf('的') < idf("小米")

True

In [54]:
def tf(word, document):
    """
    Gets the term frequency of a @word in a @document.
    """
    words = document.split()
    return sum(1 for w in words if w==word)

In [55]:
tf('银行', news_content[11])

6

In [56]:
idf('银行')

1.4550169427748936

In [64]:
def get_keywords_of_a_doucment(document):
    words = set(document.split())
    
    tfidf = [
        (w, tf(w, document) * idf(w)) for w in words
    ]
    
    tfidf = sorted(tfidf, key=lambda x:x[1], reverse=True)
    
    return tfidf

In [ ]:
get_keywords_of_a_doucment(news_content[0])

##  如何函数运行的慢，可以在juypter 上面用　% prun　来看程序的哪个地方(代码)运行的慢

In [66]:
% prun get_keywords_of_a_doucment(news_content[0])

## 词云　wordcloud
展示得好看写

github: https://github.com/amueller/word_cloud

wordcloud支持中文字体
github:
https://github.com/Computing-Intelligence/datasource
中的SourceHanSerifSC-Regular.otf

In [ ]:
import wordcloud

In [68]:
machine_new_keywords = get_keywords_of_a_doucment(news_content[101])

In [71]:
machine_new_keywords_dict = {w:score for w, score in machine_new_keywords}

In [ ]:
wc = wordcloud.WordCloud('.../SourceHanSerifSC-Regular.otf')
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(wc.generate_from_frequencies(machine_new_keywords_dict))

## The Vector Space: 向量空间。（word2vec, tfidf等等都是将信息表征成一个向量）

## 如何将信息表征成计算机可以使用的东西


tf-idf 不仅仅可以对word，也可以把整个文章弄成一个向量，再使用tf-idf。

tf-idf缺点：
    1. 无法解决同义词问题。比如将文章的“生气”改为“愤怒”，那么两个文章的tf-idf会很不一样。
    2. 无法解决带有感情色彩文章的问题。比如“否定的否定”，“否定否定再否定”。



word2vec方法是一个抽象的的概念，上一节课的仅仅是一种word2vec。今年(2019)也提出了很多的word2vec方法(比如BERT)。

# The importance of Representation 表征的重要性

表征：　
    
    如何将信息变为计算机的可以认识的信息。
    
    如何将文本变向量

ML其实只有两件事：表征＋决策

# 01:53:52  继续

某篇文章　Ｄ1: ABCD
某篇文章　Ｄ2: BAEF

则文章的tf-idf: [a,b,c,d,e,f](里面每一个值表示对应大写的tf-idf)

文章D1的tf-idf: [a1,b1,c1,d1,e1,f1]

文章D2的tf-idf: [a2,b2,c2,d2,e2,f2]

则D1和D2的相似性为：二者向量的cos余弦　cos = (v1 * v2)/(|v1| * |v2|)

cos越大，夹角越小，　表示向量越相近


# TFIDF Vectorized

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
vectorized = TfidfVectorizer()

In [ ]:
sample_num = 1000
sub_samples = new_content[:sample_num]
X = vectorized.fit_transform(sub_samples)

In [ ]:
X.shape

In [ ]:
vectorized.vocabulary_

In [ ]:
np.where(X[0].toarray())

In [5]:
import random

In [7]:
document_id_1, document_id_2, document_id_3 = random.randint(0, 1000), random.randint(0, 1000), random.randint(0, 1000)

In [ ]:
vector_of_d_1 = X[document_id_1].toarray()[0]
vector_of_d_2 = X[document_id_2].toarray()[0]    
vector_of_d_3 = X[document_id_3].toarray()[0]

In [8]:
from scipy.spatial.distance import cosine

In [9]:
def distance(v1, v2):
    return cosine(v1, v2)

In [ ]:
distance(vector_of_d_3, vector_of_d_1)

In [ ]:
distance(vector_of_d_3, vector_of_d_2)

In [ ]:
sorted(list(range(1000)), key= lambda i:distance(vector_of_d_3, X[i].toarray()[0]))

# boolean search　布尔搜索（也叫倒排索引）

优点：

    １．to process large document collections quickly.（快）
    2. to allow more flexible matching operations
    3. to allow ranked retrieval (page rank)

建立一张表
出现为１，没出现过为０

     文章１　　　文章２　　文章３　　文章４　　文章５

word1  1          1      0       0        1

word2  0          1      1       1        0

word3

word4

word5

则在搜索引擎中，输入(word1 word2) 那么应该找哪个文章？

直接　与and 即可　11001　and 01110 = 01000，则应该找文章2即可

word1 : 11001，在计算机中其实可以直接用一个int就可以表示出11001。　即我们直接写出word1:25

# 使用boolean search和tf-idf建立一个搜索引擎

# Build Search Engine

Input : Words　关键词

Output: Documnet　搜索得到的文章

In [10]:
from functools import reduce
d1, d2, d3 = {1, 2, 3}, {4, 5, 6, 3, 2}, {1, 3, 4}
from operator import and_
reduce(and_, [d1,d2,d3])

In [ ]:
word_2_id = vectorized.vocabulary_
id_2_word = {d: w for w, d in word_2_id.items()}


In [ ]:
transposed_x = X.transpace().toarray()


In [ ]:

def search_engine(query):
    """
    @query is the searched words, splited by space
    @return is the related documents which ranked by tfidf similarity
    """
    words = query.split()
    
    query_vec = vectorized.transform([' '.join(words)]).toarray()[0]
    
    candidates_ids = [word_2_id[w] for w in words]
    
    documnets_ids = [
        set(np.where(transposed_x[_id])[0] for _id in candidateds_ids)
    ]
    
    from operator import and_
    merged_documents = reduce(and_, documnets_ids)
    
    # we could know the documents which contain these words
    
    sorted_documents_id = sorted(merged_documents, key=lambda i: distance(query_vec, X[i].toarray()[0]))
    return sorted_documents_id

In [ ]:
#%%timeit

candidates_ids = search_engine('知否　绿肥　红瘦')

In [11]:
import re

In [14]:
text = """知否asadsasd替换fsfadsf掉dsad绿肥dsd它们dddadsads红瘦dsdsadsa了"""
pat = r'(知否|绿肥|红瘦)'
re.compile(pat).sub(repl="**\g<1>**", string=text)

'**知否**asadsasd替换fsfadsf掉dsad**绿肥**dsd它们dddadsads**红瘦**dsdsadsa了'

In [ ]:
def get_query_pat(query):
    return re.compile('({})'.format('|'.join(query.split())))

In [ ]:
def highlight_keywords(pat, document):
    return pat.sub(repl="**\g<1>**", string=document)

In [ ]:
highlight_keywords(get_query_pat('知否　绿肥　红瘦'), content['content'][111])

In [ ]:
def search_engine_with_pretty_print(query):
    candidates_ids = search_engine(query)
    
    for i, _id in enumerate(candidates_ids):
        print('## Search Result {}'.format(i+1))
        c = content['content'][_id]
        highlight_keywords(get_query_pat(query), c)
        print(c)

In [ ]:
search_engine_with_pretty_print('知否　绿肥　红瘦')

# PagePank

先假设　所有网站同样重要

然后，引用它的网站越多，说明这个网站越重要

In [16]:
import networkx as nx

In [17]:
import random